In [ ]:
!pip install -q transformers datasets torch Pillow bitsandbytes accelerate
#구글 드라이브 연
from google.colab import drive
drive.mount('/content/drive')

import os
from datasets import load_dataset
from PIL import Image
import json

DATA_PATH = "/content/drive/MyDrive/COCO_Caption_10k"
IMAGE_DIR = os.path.join(DATA_PATH, "images")
METADATA_FILE = os.path.join(DATA_PATH, "metadata.jsonl")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.makedirs(IMAGE_DIR, exist_ok=True)
if not os.path.exists(METADATA_FILE):
    print("데이터셋을 다운로드하고 구글 드라이브에 저장합니다. (캡션 5개 모두 활용)")

    dataset = load_dataset("lmms-lab/COCO-Caption2017", split='val', streaming=True)
    subset = dataset.take(10000)

    total_captions = 0
    with open(METADATA_FILE, 'w') as f:
        for i, example in enumerate(subset):
            if i % 500 == 0: print(f"{i}번째 이미지 처리 중... (총 캡션 수: {total_captions})")
            image = example['image'].convert("RGB")

            # 이미지는 한 번만 저장합니다.
            image_filename = example['file_name']
            image_path = os.path.join(IMAGE_DIR, image_filename)
            if not os.path.exists(image_path):
                image.save(image_path)

            for caption in example['answer']:
                # 동일한 이미지 경로에 대해 다른 캡션을 가진 메타데이터를 생성합니다.
                metadata_item = {"image_path": image_path, "caption": caption}
                f.write(json.dumps(metadata_item) + "\n")
                total_captions += 1
            # ------------------------------------

    print(f"데이터셋 준비 완료! 10000개 이미지로부터 총 {total_captions}개의 캡션 데이터가 생성되었습니다.")
else:
    print("데이터셋이 이미 구글 드라이브에 준비되어 있습니다.")


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    CLIPVisionModel,
    CLIPImageProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from PIL import Image
import json
import os
from tqdm import tqdm

In [ ]:
class Config:
    llm_model_name = "lmsys/vicuna-7b-v1.5"
    clip_model_name = "openai/clip-vit-large-patch14"

    # 데이터 경로

    DATA_PATH = "/content/drive/MyDrive/COCO_Caption_10k"
    IMAGE_DIR = os.path.join(DATA_PATH, "images")
    METADATA_FILE = os.path.join(DATA_PATH, "metadata.jsonl")

    # 훈련 반복 횟수 (요청하신 대로 변수로 지정)
    train_epochs = 7

    # 배치 사이즈 및 학습률
    batch_size = 4
    learning_rate = 1e-4

    # 저장 경로
    # 1. 초기 프로젝션 레이어 가중치를 저장할 폴더
    initial_projector_dir = "/content/drive/MyDrive/LLava_stage0_first"
    # 2. 학습 체크포인트 및 최종 결과물을 저장할 폴더
    output_dir = "/content/drive/MyDrive/llava_stage1_checkpoint"

config = Config()

# 저장 폴더가 없으면 생성
os.makedirs(config.initial_projector_dir, exist_ok=True)
os.makedirs(config.output_dir, exist_ok=True)

In [ ]:
# LLM을 4-bit로 로딩하여 VRAM 사용량을 크게 줄입니다.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# LLM (Vicuna) 및 토크나이zer 로딩
llm_model = AutoModelForCausalLM.from_pretrained(
    config.llm_model_name,
    quantization_config=quantization_config,
    #device_map="auto" # GPU에 자동으로 모델 레이어를 할당합니다.
    device_map="auto"
)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.llm_model_name, use_fast=False)
# 🧠 핵심 단계: 이미지 임베딩을 위한 특수 토큰 '<image>' 추가
IMAGE_TOKEN = "<image>"
if IMAGE_TOKEN not in tokenizer.get_added_vocab():
    # 토크나이저에 <image> 토큰을 추가합니다.
    tokenizer.add_tokens([IMAGE_TOKEN], special_tokens=True)
    # 모델의 임베딩 레이어 크기를 새 토크나이저 크기에 맞게 조정합니다.
    llm_model.resize_token_embeddings(len(tokenizer))
    print(f"'{IMAGE_TOKEN}' 토큰을 추가하고 모델 임베딩 사이즈를 조정했습니다.")

# Vicuna는 기본 padding token이 없으므로, unknown token으로 설정해줍니다.
tokenizer.pad_token = tokenizer.unk_token
IMAGE_TOKEN_ID = tokenizer.convert_tokens_to_ids(IMAGE_TOKEN)

# 이미지 인코더 (CLIP) 및 전처리기 로딩
vision_encoder = CLIPVisionModel.from_pretrained(config.clip_model_name).to(llm_model.device)
image_processor = CLIPImageProcessor.from_pretrained(config.clip_model_name)

print("✅ 모델 및 토크나이저 로딩 완료!")

In [ ]:
class LLaVADataset(Dataset):
    # def __init__(self, metadata_file, tokenizer, image_processor):
    #     self.tokenizer = tokenizer
    #     self.image_processor = image_processor
    #     self.data = []
    #     with open(metadata_file, 'r') as f:
    #         for line in f:
    #             self.data.append(json.loads(line))
    def __init__(self, metadata_file, tokenizer, image_processor):
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.data = []
        with open(metadata_file, 'r') as f:
            # 파일의 각 줄을 순회하며 데이터를 self.data에 추가
            for i, line in enumerate(f):
                # ⭐️ 핵심: 1000개의 데이터만 로드하고 중단합니다.
                if i >= 2000:
                    break
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = item['image_path']
        try:
            image = Image.open(image_path).convert("RGB")
            pixel_values = self.image_processor(image, return_tensors="pt")['pixel_values'].squeeze(0)
        except Exception:
            return self.__getitem__((idx + 1) % len(self))

        caption = item['caption']
        text_prompt = f"USER: {IMAGE_TOKEN}\nDescribe this image.\nASSISTANT: {caption}{self.tokenizer.eos_token}"
        tokenized_output = self.tokenizer(text_prompt, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
        input_ids = tokenized_output['input_ids'].squeeze(0)

        labels = input_ids.clone()
        prompt_part = text_prompt.split("ASSISTANT:")[0] + "ASSISTANT:"
        prompt_tokens = self.tokenizer(prompt_part, return_tensors="pt")['input_ids']
        mask_until_index = prompt_tokens.shape[1]
        labels[:mask_until_index] = -100
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values.to(llm_model.dtype),
            "input_ids": input_ids,
            "labels": labels
        }

# =================================================================================
# 5. LLaVA 모델 아키텍처 정의 및 초기 가중치 저장
# =================================================================================
class LLaVAModel(nn.Module):
    def __init__(self, vision_encoder, llm_model, image_token_id):
        super().__init__()
        self.vision_encoder = vision_encoder
        self.llm_model = llm_model
        self.image_token_id = image_token_id

        # 1. 프로젝션 레이어 정의: CLIP 출력 차원 -> LLM 입력 차원
        # n*m : m 차원에서 n 차원으로
        self.projection_layer = nn.Linear(
            self.vision_encoder.config.hidden_size,
            self.llm_model.config.hidden_size
        )

        for param in self.vision_encoder.parameters(): param.requires_grad = False
        for param in self.llm_model.parameters(): param.requires_grad = False
        for param in self.projection_layer.parameters(): param.requires_grad = True

    def forward(self, pixel_values, input_ids, labels=None, **kwargs):
        image_outputs = self.vision_encoder(pixel_values=pixel_values, return_dict=True)
        image_features = image_outputs.pooler_output
        projected_image_features = self.projection_layer(image_features)
        word_embeddings = self.llm_model.get_input_embeddings()(input_ids)

        for i in range(input_ids.shape[0]):
            image_token_index = (input_ids[i] == self.image_token_id).nonzero(as_tuple=True)[0]
            if image_token_index.numel() > 0:
                word_embeddings[i, image_token_index] = projected_image_features[i].to(word_embeddings.dtype)

        return self.llm_model(inputs_embeds=word_embeddings, labels=labels, return_dict=True)

# 모델 인스턴스 생성
model = LLaVAModel(vision_encoder, llm_model, IMAGE_TOKEN_ID)
final_projector_path = os.path.join(config.output_dir, "final_projection_layer.pt")

# 만약 이전에 학습하고 저장한 가중치 파일이 존재하면, 해당 가중치를 불러옵니다.
if os.path.exists(final_projector_path):
    print(f"✅ 이전에 학습된 프로젝션 레이어 가중치를 {final_projector_path}에서 불러옵니다.")
    # torch.load로 가중치 파일을 읽어온 후, model.projection_layer.load_state_dict로 적용합니다.
    model.projection_layer.load_state_dict(torch.load(final_projector_path))
else:
    # 학습된 가중치 파일이 없으면 (가장 처음 실행하는 경우), 새로운 레이어로 시작합니다.
    print("🔵 학습된 가중치 파일이 없으므로, 새로운 프로젝션 레이어로 시작합니다.")
    # 이 경우에만 초기 가중치를 저장하도록 할 수 있습니다.
    initial_projector_path = os.path.join(config.initial_projector_dir, "initial_projection_layer.pt")
    torch.save(model.projection_layer.state_dict(), initial_projector_path)
    print(f"✨ 초기 프로젝션 레이어 가중치를 {initial_projector_path}에 저장했습니다.")

#
##if not os.path.exists(config.initial_projector_dir):
  #학습 전, 초기 프로젝션 레이어 가중치를 파일로 저장
  #initial_projector_path = os.path.join(config.initial_projector_dir, "initial_projection_layer.pt")
  #torch.save(model.projection_layer.state_dict(), initial_projector_path)
  #print(f"✅ 초기 프로젝션 레이어 가중치를 {initial_projector_path}에 저장했습니다.")


In [ ]:
# 데이터셋 인스턴스 생성
train_dataset = LLaVADataset(config.METADATA_FILE, tokenizer, image_processor)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=config.output_dir,
    num_train_epochs=config.train_epochs,
    per_device_train_batch_size=config.batch_size,
    gradient_accumulation_steps=8,
    learning_rate=config.learning_rate,
    logging_steps=20,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    remove_unused_columns=False,
    report_to="none",
)

# Trainer 정의
trainer = Trainer( #허깅페이스 함수 사용
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# 학습 시작!
trainer.train()
print(f"{config.train_epochs} 번 최종 학습 종료")

# 학습 완료 후, 최종 프로젝션 레이어 가중치를 파일로 저장
final_projector_path = os.path.join(config.output_dir, "final_projection_layer.pt")
torch.save(model.projection_layer.state_dict(), final_projector_path)



# 실습 결과
### 1. Projection Layer 크기

### 1000개로 20번 2000개로 20번

In [ ]:

# 1. 프로젝션 레이어의 가중치(weight) 행렬의 크기를 가져옵니다.
projection_layer_shape = model.projection_layer.weight.shape

#    입력 차원: CLIP Vision Encoder의 출력 차원
#    출력 차원: LLM (Vicuna)의 입력 임베딩 차원
input_dim = model.vision_encoder.config.hidden_size
output_dim = model.llm_model.config.hidden_size

print("="*50)
print("🔍 Projection Layer 크기 정보")
print(f"  - 레이어의 전체 가중치 행렬 크기 (shape): {projection_layer_shape}")
print(f"  - 입력 차원 (Input Dimension): {input_dim} (CLIP Vision Encoder의 출력)")
print(f"  - 출력 차원 (Output Dimension): {output_dim} (Vicuna LLM의 입력)")
print("="*50)

# =================================================================================

### 2. 임의의 데이터 셋에 대해 정답 캡션과 임베딩 차이 비교

### 학습에 사용하지 않음 500개 이미지-캡션 데이터셋 사용



In [ ]:
import torch.nn.functional as F
all_data = []
with open(config.METADATA_FILE, 'r') as f:
    for line in f:
        all_data.append(json.loads(line))

eval_data = all_data[-500:]


print("\n" + "="*50)
print("🧪 평가를 시작합니다: 이미지-캡션 코사인 유사도 측정")
print("="*50)

# 평가를 위해 모델을 eval 모드로 설정
model.eval()

# LLM의 단어 임베딩 레이어를 가져옵니다.
word_embedding_layer = model.llm_model.get_input_embeddings()

device = model.llm_model.device
total_similarity = 0.0
evaluated_count = 0

try:
    # final_projector_path 변수는 스크립트 상단에 정의되어 있어야 합니다.
    print(f"🔵 '{final_projector_path}'에서 최종 프로젝션 레이어 가중치를 불러옵니다...")
    # 지정된 경로에서 가중치 파일을 불러와 현재 모델의 projection_layer에 적용합니다.
    model.projection_layer.load_state_dict(torch.load(final_projector_path))
    print("✅ 최종 가중치를 성공적으로 모델에 적용했습니다.")
except FileNotFoundError:
    print(f"❌ 에러: 최종 가중치 파일 '{final_projector_path}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")
    # 파일이 없으면 평가를 중단합니다.
    exit()


# 평가 중에는 그래디언트 계산이 필요 없으므로 torch.no_grad() 사용
with torch.no_grad():
    # 평가 데이터(마지막 500개)를 하나씩 순회
    for item in tqdm(eval_data, desc="Evaluating Cosine Similarity"):
        try:
            # 1. 이미지 처리 및 이미지 임베딩 생성
            image_path = item['image_path']
            image = Image.open(image_path).convert("RGB")

            # 이미지를 CLIP 프로세서로 전처리하고 모델과 동일한 디바이스로 이동
            pixel_values = image_processor(image, return_tensors="pt")['pixel_values'].to(device, dtype=torch.float16)

            # Vision Encoder를 통과시켜 이미지 피처 추출
            image_outputs = model.vision_encoder(pixel_values=pixel_values, return_dict=True)
            image_features = image_outputs.pooler_output

            # 학습된 프로젝션 레이어를 통과시켜 최종 이미지 임베딩 생성
            image_embedding = model.projection_layer(image_features) # shape: (1, 4096)

            # 2. 캡션 처리 및 캡션 임베딩 생성
            caption = item['caption']

            # 캡션을 토크나이징하고 모델과 동일한 디바이스로 이동
            input_ids = tokenizer(caption, return_tensors='pt')['input_ids'].to(device)

            # 토큰 ID를 단어 임베딩으로 변환
            caption_token_embeddings = word_embedding_layer(input_ids) # shape: (1, sequence_length, 4096)

            # 문장 전체를 대표하는 하나의 벡터를 얻기 위해 평균을 계산 (패딩 토큰은 제외)
            caption_embedding = caption_token_embeddings.mean(dim=1) # shape: (1, 4096)

            # 3. 코사인 유사도 계산
            similarity = F.cosine_similarity(image_embedding, caption_embedding)
            total_similarity += similarity.item()
            evaluated_count += 1

        except Exception as e:
            # 파일이 깨져있거나 문제가 있는 경우 건너뜀
            print(f"Skipping {item.get('image_path', 'unknown image')} due to error: {e}")
            continue

# 최종 평균 코사인 유사도 계산 및 출력
if evaluated_count > 0:
    average_similarity = total_similarity / evaluated_count
    print("\n" + "="*50)
    print("✨ 평가 결과 ✨")
    print(f"  - 평가된 데이터 개수: {evaluated_count} / {len(eval_data)}")
    print(f"  - 평균 코사인 유사도: {average_similarity:.4f}")
    print("="*50)
else:
    print("평가를 진행할 수 있는 데이터가 없습니다.")

### 3. 초기 행렬 값과 훈련 후 행렬 값 동일한 임베딩 변환 차이 비교

In [ ]:
import torch.nn.functional as F
import json
from PIL import Image
from tqdm import tqdm
import os

all_data = []
# config 변수는 스크립트 상단에 정의되어 있다고 가정합니다.
with open(config.METADATA_FILE, 'r') as f:
    for line in f:
        all_data.append(json.loads(line))

eval_data = all_data[-500:]


print("\n" + "="*50)
print("🧪 초기 프로젝션 레이어로 평가를 시작합니다.")
print("="*50)

# --- [요청하신 수정 사항] 초기 프로젝션 레이어 가중치 불러오기 ---
# 스크립트 상단에 정의된 config 변수를 사용하여 초기 가중치 파일 경로를 정의합니다.
initial_projector_path = os.path.join(config.initial_projector_dir, "initial_projection_layer.pt")

try:
    print(f"🔵 '{initial_projector_path}'에서 초기 프로젝션 레이어 가중치를 불러옵니다...")
    # 지정된 경로에서 가중치 파일을 불러와 현재 모델의 projection_layer에 적용합니다.
    model.projection_layer.load_state_dict(torch.load(initial_projector_path))
    print("✅ 초기 가중치를 성공적으로 모델에 적용했습니다.")
except FileNotFoundError:
    print(f"❌ 에러: 초기 가중치 파일 '{initial_projector_path}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")
    # 파일이 없으면 평가를 중단합니다.
    exit()
# -------------------------------------------------------------------


# 평가를 위해 모델을 eval 모드로 설정
model.eval()

# LLM의 단어 임베딩 레이어를 가져옵니다.
word_embedding_layer = model.llm_model.get_input_embeddings()

device = model.llm_model.device
total_similarity = 0.0
evaluated_count = 0

# 평가 중에는 그래디언트 계산이 필요 없으므로 torch.no_grad() 사용
with torch.no_grad():
    # 평가 데이터(마지막 500개)를 하나씩 순회
    for item in tqdm(eval_data, desc="Evaluating with Initial Weights"):
        try:
            # 1. 이미지 처리 및 이미지 임베딩 생성
            image_path = item['image_path']
            image = Image.open(image_path).convert("RGB")

            # 이미지를 CLIP 프로세서로 전처리하고 모델과 동일한 디바이스로 이동
            pixel_values = image_processor(image, return_tensors="pt")['pixel_values'].to(device, dtype=torch.float16)

            # Vision Encoder를 통과시켜 이미지 피처 추출
            image_outputs = model.vision_encoder(pixel_values=pixel_values, return_dict=True)
            image_features = image_outputs.pooler_output

            # 학습된 프로젝션 레이어를 통과시켜 최종 이미지 임베딩 생성
            image_embedding = model.projection_layer(image_features) # shape: (1, 4096)

            # 2. 캡션 처리 및 캡션 임베딩 생성
            caption = item['caption']

            # 캡션을 토크나이징하고 모델과 동일한 디바이스로 이동
            input_ids = tokenizer(caption, return_tensors='pt')['input_ids'].to(device)

            # 토큰 ID를 단어 임베딩으로 변환
            caption_token_embeddings = word_embedding_layer(input_ids) # shape: (1, sequence_length, 4096)

            # 문장 전체를 대표하는 하나의 벡터를 얻기 위해 평균을 계산 (패딩 토큰은 제외)
            caption_embedding = caption_token_embeddings.mean(dim=1) # shape: (1, 4096)

            # 3. 코사인 유사도 계산
            similarity = F.cosine_similarity(image_embedding, caption_embedding)
            total_similarity += similarity.item()
            evaluated_count += 1

        except Exception as e:
            # 파일이 깨져있거나 문제가 있는 경우 건너뜀
            print(f"Skipping {item.get('image_path', 'unknown image')} due to error: {e}")
            continue

# 최종 평균 코사인 유사도 계산 및 출력
if evaluated_count > 0:
    average_similarity = total_similarity / evaluated_count
    print("\n" + "="*50)
    print("✨ 평가 결과 (초기 가중치) ✨")
    print(f"  - 평가된 데이터 개수: {evaluated_count} / {len(eval_data)}")
    print(f"  - 평균 코사인 유사도: {average_similarity:.4f}")
    print("="*50)
else:
    print("평가를 진행할 수 있는 데이터가 없습니다.")

## 1. Dalle-2: prior CLIP 텍스트 임베딩을 선형 변환하는 것과 이미지 임베딩 생성하는 것의 성능 차이 비교

## 2. Latent Space(잠재공간) 탐색 이미지1,이미지2를 잠재 공간에 넣고 잠재 공간에서 벡터1->벡터2로 하나씩 디코더에 넣으면서 이미지 변화 추이 살펴보